In [49]:
import os
import getpass
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from langchain_community.tools import TavilySearchResults
from newspaper import Article, Config
import nltk

In [ ]:
# Funciones
def extraer_contenido_newspaper(url):
    try:
        config = Config()
        config.browser_user_agent = (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/85.0.4183.102 Safari/537.36"
        )
        article = Article(url, config=config)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return f"Error al extraer contenido {e}"
def obtener_titulo_completo(url):
    try:
        respuesta = requests.get(url, timeout=10)
        if respuesta.ok:
            soup = BeautifulSoup(respuesta.text, "html.parser")
            titulo = soup.title.string if soup.title else "Título no encontrado"
            return titulo.strip()
        else:
            return "Error al obtener la página"
    except Exception as e:
        return f"Error: {e}"

def formatear_texto_continuo(texto):
    return " ".join(texto.split())



In [ ]:
# ---------------------------
# Configuración de la API
# ---------------------------
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
tool = TavilySearchResults(
    max_results=100,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True
)

In [ ]:
# ----------------------
# Definición del subtema
# ----------------------
variations = [
    # Subtema 1:
    # "¿Cómo han reaccionado los sectores sociales y económicos ante las decisiones políticas y económicas de Dina Boluarte?"
    "¿Cómo han respondido los sectores sociales y económicos a las decisiones políticas y económicas de Dina Boluarte?",
    "¿Qué respuestas han mostrado los sectores sociales y económicos frente a las medidas políticas y económicas adoptadas por Dina Boluarte?",
    "¿De qué forma han reaccionado los ámbitos social y económico ante las políticas implementadas por Dina Boluarte?",
    "¿Cuál ha sido la reacción de los sectores social y económico a las decisiones gubernamentales de Dina Boluarte?",
    "¿Cómo se han manifestado las respuestas de la sociedad y del sector económico ante las decisiones políticas y económicas de Dina Boluarte?",
    "¿Qué actitud han tomado los sectores sociales y económicos ante las decisiones en política y economía de Dina Boluarte?",
    "¿Cómo han respondido los diversos sectores sociales y económicos a las medidas adoptadas por el gobierno de Dina Boluarte?",
    "¿Qué reacción se observa en los sectores social y económico respecto a las políticas económicas y políticas de Dina Boluarte?",
    "¿Cómo han reaccionado la sociedad y los sectores económicos ante las decisiones de política y economía de Dina Boluarte?",
    "¿De qué manera se han manifestado las reacciones de los sectores social y económico frente a las decisiones de Dina Boluarte en materia de política y economía?",
    "¿Qué opiniones expresan los sectores sociales y económicos ante las decisiones políticas y económicas del gobierno de Dina Boluarte?",
    "¿Cómo han actuado los sectores sociales y económicos en respuesta a las políticas implementadas por Dina Boluarte?",
    "¿Qué impacto han tenido las decisiones políticas y económicas de Dina Boluarte en la reacción de los sectores social y económico?",
    "¿Cómo han reaccionado los diferentes sectores sociales y económicos ante las medidas de política y economía adoptadas por Dina Boluarte?",
    "¿Cuál ha sido la respuesta de la opinión social y del sector económico a las decisiones políticas y económicas de Dina Boluarte?",
    "¿Cómo han percibido los sectores social y económico las decisiones en materia de política y economía de Dina Boluarte?",
    "¿Qué reacciones se han evidenciado en los sectores sociales y económicos ante las políticas y decisiones económicas de Dina Boluarte?",
    "¿De qué forma se ha manifestado la respuesta de los sectores social y económico ante las medidas adoptadas por Dina Boluarte?",
    "¿Qué impresiones han generado en los sectores sociales y económicos las decisiones políticas y económicas de Dina Boluarte?",
    "¿Cómo han reaccionado los grupos sociales y económicos frente a las decisiones adoptadas en política y economía por Dina Boluarte?",
    "¿Qué respuesta han mostrado los sectores social y económico ante las medidas y decisiones implementadas por Dina Boluarte?",
    "¿Cómo se ha reflejado en los sectores sociales y económicos la respuesta a las políticas económicas y decisiones de Dina Boluarte?",
    "¿De qué manera han contestado los sectores sociales y económicos a las decisiones políticas y económicas de Dina Boluarte?",
    "¿Qué reacción han tenido los sectores social y económico frente a las decisiones gubernamentales de Dina Boluarte?",
    "¿Cómo han respondido los sectores de la sociedad y de la economía ante las medidas adoptadas por Dina Boluarte?",
    "¿Qué percepciones se han evidenciado en los sectores sociales y económicos respecto a las políticas y decisiones de Dina Boluarte?",
    "¿De qué forma se han expresado las reacciones de los sectores social y económico ante las políticas implementadas por Dina Boluarte?",
    "¿Cómo han manifestado los sectores sociales y económicos su respuesta a las decisiones en materia de política y economía de Dina Boluarte?",
    "¿Qué actitudes han emergido en los sectores social y económico ante las decisiones políticas y económicas de Dina Boluarte?",
    "¿Cómo se han mostrado las respuestas de los sectores sociales y económicos frente a las políticas y decisiones económicas de Dina Boluarte?",

    # Subtema 2:
    # "¿Cómo ha manejado Dina Boluarte las protestas y manifestaciones sociales en comparación con gobiernos anteriores?"
    "¿Cómo ha gestionado Dina Boluarte las protestas y manifestaciones sociales en contraste con administraciones previas?",
    "¿De qué manera ha abordado Dina Boluarte las manifestaciones sociales comparado con gobiernos anteriores?",
    "¿Qué diferencias existen en el manejo de protestas por parte de Dina Boluarte en comparación con gobiernos previos?",
    "¿Cómo ha respondido el gobierno de Dina Boluarte ante las protestas, y cómo se compara con la respuesta de gobiernos anteriores?",
    "¿Cuál es el enfoque de Dina Boluarte para gestionar las manifestaciones sociales versus el de administraciones previas?",
    "¿Cómo contrasta el manejo de las protestas sociales de Dina Boluarte con el de gobiernos anteriores?",
    "¿En qué se diferencia la estrategia de Dina Boluarte para manejar las manifestaciones sociales respecto a administraciones anteriores?",
    "¿Qué método ha empleado Dina Boluarte para enfrentar las protestas sociales en comparación con gobiernos previos?",
    "¿Cómo se distingue la respuesta de Dina Boluarte ante las protestas y manifestaciones sociales frente a administraciones anteriores?",
    "¿Qué críticas se han hecho al manejo de las protestas por parte de Dina Boluarte en comparación con gobiernos anteriores?",
    "¿Cómo evalúan las reacciones a las protestas la gestión de Dina Boluarte versus la de gobiernos anteriores?",
    "¿Qué diferencias en el manejo de manifestaciones sociales se observan entre la administración de Dina Boluarte y gobiernos previos?",
    "¿Cómo ha implementado Dina Boluarte sus estrategias contra las protestas comparado con administraciones anteriores?",
    "¿Cuál es la diferencia en la forma de manejar las protestas sociales entre el gobierno de Dina Boluarte y los anteriores?",
    "¿Cómo ha sido la gestión de las manifestaciones sociales bajo el mandato de Dina Boluarte en comparación con gobiernos anteriores?",
    "¿Qué particularidades tiene el manejo de protestas por parte de Dina Boluarte frente a lo realizado por gobiernos anteriores?",
    "¿En qué se diferencian las respuestas a las protestas sociales de Dina Boluarte en comparación con las de administraciones previas?",
    "¿Qué enfoques distintos se observan en el manejo de manifestaciones sociales entre Dina Boluarte y gobiernos anteriores?",
    "¿Cómo ha confrontado Dina Boluarte las protestas y manifestaciones sociales, y cómo se compara su método con el de gobiernos previos?",
    "¿Qué estrategias se destacan en el manejo de protestas de Dina Boluarte versus las de administraciones anteriores?",
    "¿Cómo difiere el tratamiento de las protestas y manifestaciones sociales en el gobierno de Dina Boluarte respecto a gobiernos anteriores?",
    "¿Qué diferencias se evidencian en la gestión de protestas sociales entre la administración de Dina Boluarte y las anteriores?",
    "¿Cómo ha reaccionado el gobierno de Dina Boluarte ante las manifestaciones en comparación con las respuestas de gobiernos previos?",
    "¿Qué aspectos resaltan en el manejo de protestas de Dina Boluarte frente a los de administraciones anteriores?",
    "¿En qué aspectos se diferencia la respuesta del gobierno de Dina Boluarte a las manifestaciones sociales de la de gobiernos anteriores?",
    "¿Cómo ha diferido la estrategia de Dina Boluarte para enfrentar las protestas sociales comparada con la de gobiernos anteriores?",
    "¿Qué contraste se puede observar en el manejo de las protestas sociales entre la administración de Dina Boluarte y la de gobiernos previos?",
    "¿Cómo se compara la respuesta ante manifestaciones sociales de Dina Boluarte con la de gobiernos anteriores?",
    "¿Qué matices tiene la gestión de protestas sociales por parte de Dina Boluarte en comparación con administraciones previas?",
    "¿En qué se ha distinguido el manejo de protestas y manifestaciones sociales por Dina Boluarte respecto a gobiernos anteriores?",

    # Subtema 3:
    # "¿Qué impacto ha tenido la política exterior de Dina Boluarte en la imagen internacional del Perú?"
    "¿Cómo ha influido la política exterior de Dina Boluarte en la percepción internacional del Perú?",
    "¿Qué efecto ha tenido la gestión en política exterior de Dina Boluarte sobre la imagen global del Perú?",
    "¿En qué medida la política exterior de Dina Boluarte ha afectado la reputación internacional del Perú?",
    "¿Cómo se ha transformado la imagen internacional del Perú a raíz de la política exterior de Dina Boluarte?",
    "¿Qué cambios en la percepción global del Perú se atribuyen a la política exterior de Dina Boluarte?",
    "¿Cuál es el impacto de la política exterior de Dina Boluarte en la proyección internacional del Perú?",
    "¿Cómo evalúan los analistas internacionales la política exterior de Dina Boluarte y su efecto en la imagen del Perú?",
    "¿Qué repercusiones ha tenido la política exterior de Dina Boluarte en la percepción del Perú a nivel mundial?",
    "¿Cómo ha modificado la política exterior de Dina Boluarte la imagen internacional del Perú?",
    "¿Qué transformaciones en la reputación internacional del Perú se observan tras la política exterior de Dina Boluarte?",
    "¿Qué impacto se ha generado en la imagen global del Perú a partir de la política exterior implementada por Dina Boluarte?",
    "¿De qué forma ha influido la política exterior de Dina Boluarte en la imagen del Perú en el escenario internacional?",
    "¿Qué cambios en la percepción internacional del Perú se relacionan con la política exterior de Dina Boluarte?",
    "¿Cómo se refleja en la imagen internacional del Perú la política exterior adoptada por Dina Boluarte?",
    "¿Qué efecto han tenido las decisiones en política exterior de Dina Boluarte sobre la imagen internacional del Perú?",
    "¿En qué medida ha impactado la política exterior de Dina Boluarte en la percepción global del Perú?",
    "¿Qué evidencias muestran el impacto de la política exterior de Dina Boluarte en la reputación del Perú a nivel internacional?",
    "¿Cómo ha repercutido la política exterior de Dina Boluarte en la imagen del Perú en la comunidad internacional?",
    "¿Qué transformaciones en la proyección internacional del Perú se han vinculado con la política exterior de Dina Boluarte?",
    "¿Cómo ha afectado la política exterior de Dina Boluarte la percepción del Perú en el ámbito internacional?",
    "¿Qué impacto tiene la política exterior de Dina Boluarte en la imagen global del Perú según los expertos?",
    "¿Cómo ha cambiado la imagen internacional del Perú debido a la política exterior de Dina Boluarte?",
    "¿Qué implicaciones tiene la política exterior de Dina Boluarte en la percepción internacional del Perú?",
    "¿En qué forma la política exterior de Dina Boluarte ha moldeado la reputación internacional del Perú?",
    "¿Qué consecuencias ha tenido la política exterior de Dina Boluarte en la imagen del Perú en el mundo?",
    "¿Cómo se ha visto afectada la proyección internacional del Perú por la política exterior de Dina Boluarte?",
    "¿Qué impacto observable tiene la política exterior de Dina Boluarte en la imagen global del Perú?",
    "¿Cómo se correlaciona la política exterior de Dina Boluarte con la percepción internacional del Perú?",
    "¿Qué cambios en la imagen del Perú a nivel internacional se pueden atribuir a la política exterior de Dina Boluarte?",
    "¿De qué manera la política exterior de Dina Boluarte ha influido en la reputación global del Perú?",

    # Subtema 4:
    # "¿Qué casos de corrupción en programas sociales, como Qali Warma, y su relación con el gobierno de Dina Boluarte?"
    "¿Qué casos de corrupción en programas sociales, como Qali Warma, se han vinculado al gobierno de Dina Boluarte?",
    "¿Cuáles son los escándalos de corrupción en programas sociales, tales como Qali Warma, relacionados con el gobierno de Dina Boluarte?",
    "¿Qué evidencias de corrupción en programas sociales, incluyendo Qali Warma, han salido a la luz durante la administración de Dina Boluarte?",
    "¿En qué casos se ha documentado corrupción en programas sociales, como Qali Warma, y cuál ha sido la implicación del gobierno de Dina Boluarte?",
    "¿Qué irregularidades se han denunciado en programas sociales como Qali Warma vinculadas al gobierno de Dina Boluarte?",
    "¿Qué ejemplos de corrupción en programas sociales, tales como Qali Warma, se han atribuido a la gestión de Dina Boluarte?",
    "¿Cómo se han expuesto los casos de corrupción en programas sociales como Qali Warma en el contexto del gobierno de Dina Boluarte?",
    "¿Qué investigaciones han revelado casos de corrupción en programas sociales, incluyendo Qali Warma, y su relación con el gobierno de Dina Boluarte?",
    "¿Qué vínculos se han establecido entre los casos de corrupción en programas sociales, como Qali Warma, y la administración de Dina Boluarte?",
    "¿Cuáles son los principales casos de corrupción en programas sociales, como Qali Warma, que afectan la imagen del gobierno de Dina Boluarte?",
    "¿Qué denuncias de corrupción en programas sociales, tales como Qali Warma, se han reportado en relación al gobierno de Dina Boluarte?",
    "¿Cómo se ha manejado la corrupción en programas sociales, por ejemplo Qali Warma, en el marco de la administración de Dina Boluarte?",
    "¿Qué irregularidades en programas sociales como Qali Warma han implicado al gobierno de Dina Boluarte en casos de corrupción?",
    "¿Qué implicaciones tiene la corrupción detectada en programas sociales como Qali Warma en el gobierno de Dina Boluarte?",
    "¿Qué casos de corrupción relacionados con programas sociales, especialmente Qali Warma, han sido señalados en el contexto del gobierno de Dina Boluarte?",
    "¿Cómo se ha evidenciado la corrupción en programas sociales como Qali Warma en la administración de Dina Boluarte?",
    "¿Qué ejemplos concretos de corrupción en programas sociales, como Qali Warma, se han asociado al gobierno de Dina Boluarte?",
    "¿Qué investigaciones periodísticas han descubierto casos de corrupción en programas sociales, como Qali Warma, vinculados a Dina Boluarte?",
    "¿Cómo se relacionan los casos de corrupción en programas sociales, como Qali Warma, con la gestión del gobierno de Dina Boluarte?",
    "¿Qué evidencias respaldan los casos de corrupción en programas sociales, tales como Qali Warma, en el contexto del gobierno de Dina Boluarte?",
    "¿Qué aspectos de corrupción en programas sociales como Qali Warma han sido atribuidos a la gestión de Dina Boluarte?",
    "¿Cómo ha influido la corrupción en programas sociales, por ejemplo Qali Warma, en la percepción del gobierno de Dina Boluarte?",
    "¿Qué denuncias y pruebas se han presentado sobre corrupción en programas sociales, como Qali Warma, durante el gobierno de Dina Boluarte?",
    "¿Qué casos emblemáticos de corrupción en programas sociales, como Qali Warma, se han relacionado con la administración de Dina Boluarte?",
    "¿De qué manera la corrupción en programas sociales, tales como Qali Warma, ha impactado al gobierno de Dina Boluarte?",
    "¿Qué irregularidades han sido detectadas en programas sociales como Qali Warma y cómo se vinculan al gobierno de Dina Boluarte?",
    "¿Cómo se ha revelado la corrupción en programas sociales, incluyendo Qali Warma, en relación con el gobierno de Dina Boluarte?",
    "¿Qué investigaciones recientes han destacado casos de corrupción en programas sociales, como Qali Warma, y su conexión con Dina Boluarte?",
    "¿Qué factores han contribuido a los casos de corrupción en programas sociales, como Qali Warma, en el marco del gobierno de Dina Boluarte?",
    "¿Cómo se manifiesta la corrupción en programas sociales, por ejemplo Qali Warma, y cuál es su relación con la administración de Dina Boluarte?"
]


In [ ]:
variations_Tema1 = [
    "¿Cómo han reaccionado los sectores sociales y económicos ante las medidas políticas y económicas implementadas por Dina Boluarte?",
    "Opiniones de los sectores sociales y económicos sobre las políticas de Dina Boluarte",
    "Reacciones de la sociedad y del ámbito económico frente a las decisiones de Dina Boluarte",
    "¿Cuál ha sido la respuesta de los sectores sociales y económicos ante las acciones políticas y económicas de Dina Boluarte?",
    "Impacto en los sectores sociales y económicos de las decisiones en política y economía de Dina Boluarte",
    "Efectos de las medidas económicas y políticas de Dina Boluarte en los sectores social y económico",
    "Análisis de la respuesta de la sociedad y el sector económico a las políticas de Dina Boluarte",
    "¿Qué opinan los diferentes sectores sociales y económicos sobre las políticas implementadas por Dina Boluarte?",
    "Evaluación de la reacción de los ámbitos social y económico frente a las decisiones de Dina Boluarte",
    "Perspectivas de los sectores social y económico sobre el manejo gubernamental de Dina Boluarte",
    "Reacción de la opinión pública y del sector económico ante las decisiones políticas de Dina Boluarte",
    "Análisis de las repercusiones en los sectores sociales y económicos derivadas de las políticas de Dina Boluarte",
    "¿Cómo ha influido la gestión de Dina Boluarte en la percepción de los sectores sociales y económicos?",
    "Impacto de las decisiones gubernamentales de Dina Boluarte en la sociedad y el ámbito económico",
    "¿Qué cambios en la actitud de los sectores sociales y económicos se observan ante las políticas de Dina Boluarte?"
]

variations_Tema2 = [
    "¿De qué manera ha respondido Dina Boluarte a las protestas y manifestaciones sociales en contraste con administraciones previas?",
    "Comparado con gobiernos anteriores, ¿cómo ha gestionado Dina Boluarte las manifestaciones sociales?",
    "¿Cuál ha sido el enfoque de Dina Boluarte para enfrentar las protestas, y en qué se diferencia del de gobiernos anteriores?",
    "¿En qué aspectos difiere la respuesta de Dina Boluarte ante las manifestaciones sociales de la de administraciones previas?",
    "¿Qué estrategias ha empleado el gobierno de Dina Boluarte para manejar las protestas en comparación con gobiernos anteriores?",
    "¿Cómo evalúan los sectores sociales la gestión de las protestas por parte de Dina Boluarte frente a la de gobiernos anteriores?",
    "¿Qué diferencias se observan en el manejo de las manifestaciones sociales entre la administración de Dina Boluarte y la de gobiernos anteriores?",
    "¿Cómo ha abordado Dina Boluarte las protestas en su gobierno, en comparación con la respuesta de administraciones anteriores?",
    "¿En qué se diferencia el manejo de las protestas por parte de Dina Boluarte del de gobiernos anteriores?",
    "¿De qué forma ha enfrentado el gobierno de Dina Boluarte las manifestaciones sociales en contraste con administraciones previas?",
    "¿Qué particularidades presenta la respuesta de Dina Boluarte a las protestas en comparación con la de gobiernos anteriores?",
    "¿Cómo se ha diferenciado la estrategia de Dina Boluarte para manejar las manifestaciones sociales frente a la adoptada por gobiernos anteriores?",
    "¿Qué críticas se han generado sobre la gestión de protestas de Dina Boluarte en comparación con las de administraciones previas?",
    "¿Cómo varía la respuesta del gobierno de Dina Boluarte ante las manifestaciones sociales respecto a la de gobiernos anteriores?",
    "¿En qué medida el manejo de protestas y manifestaciones sociales por parte de Dina Boluarte se distingue del de administraciones previas?"
]

variations_Tema3 = [
    "¿Cómo ha influido la política exterior de Dina Boluarte en la percepción internacional del Perú?",
    "¿Qué efecto ha tenido la gestión en política exterior de Dina Boluarte sobre la imagen del Perú a nivel global?",
    "¿En qué medida la política exterior de Dina Boluarte ha afectado la reputación internacional del Perú?",
    "¿Cómo se ha transformado la imagen internacional del Perú tras la política exterior implementada por Dina Boluarte?",
    "¿Qué cambios en la imagen global del Perú se atribuyen a la política exterior de Dina Boluarte?",
    "¿Cuál es el impacto de la política exterior de Dina Boluarte en la percepción del Perú en el escenario internacional?",
    "¿Cómo evalúan los expertos internacionales la política exterior de Dina Boluarte y su efecto en la imagen del Perú?",
    "¿Qué repercusiones ha tenido la política exterior de Dina Boluarte en la proyección internacional del Perú?",
    "¿Cómo ha modificado la política exterior de Dina Boluarte la imagen internacional del Perú?",
    "¿Qué transformaciones en la percepción internacional del Perú se observan a raíz de la política exterior de Dina Boluarte?",
    "¿Qué impacto han tenido las decisiones en política exterior de Dina Boluarte en la reputación global del Perú?",
    "¿De qué forma la política exterior de Dina Boluarte ha moldeado la imagen internacional del Perú?",
    "¿Cuáles son los efectos de la política exterior de Dina Boluarte sobre la imagen y reputación internacional del Perú?",
    "¿Cómo ha repercutido la política exterior de Dina Boluarte en la imagen global del Perú?",
    "¿Qué influencia ha ejercido la política exterior de Dina Boluarte en la percepción internacional del Perú?"
]

variations_Tema4 = [
    "¿Qué casos de corrupción en programas sociales, como Qali Warma, se han vinculado al gobierno de Dina Boluarte?",
    "¿Cuáles son los escándalos de corrupción en programas sociales, tales como Qali Warma, relacionados con el gobierno de Dina Boluarte?",
    "¿Qué evidencias de corrupción en programas sociales, incluyendo Qali Warma, han salido a la luz durante la administración de Dina Boluarte?",
    "¿En qué casos se ha documentado corrupción en programas sociales, como Qali Warma, y cuál ha sido la implicación del gobierno de Dina Boluarte?",
    "¿Qué irregularidades se han denunciado en programas sociales como Qali Warma vinculadas al gobierno de Dina Boluarte?",
    "¿Qué ejemplos de corrupción en programas sociales, tales como Qali Warma, se han atribuido a la gestión de Dina Boluarte?",
    "¿Cómo se han expuesto los casos de corrupción en programas sociales como Qali Warma en el contexto del gobierno de Dina Boluarte?",
    "¿Qué investigaciones han revelado casos de corrupción en programas sociales, incluyendo Qali Warma, y su relación con el gobierno de Dina Boluarte?",
    "¿Qué vínculos se han establecido entre los casos de corrupción en programas sociales, como Qali Warma, y la administración de Dina Boluarte?",
    "¿Cuáles son los principales casos de corrupción en programas sociales, como Qali Warma, que afectan la imagen del gobierno de Dina Boluarte?",
    "¿Qué denuncias de corrupción en programas sociales, tales como Qali Warma, se han reportado en relación al gobierno de Dina Boluarte?",
    "¿Cómo se ha manejado la corrupción en programas sociales, por ejemplo Qali Warma, en el marco de la administración de Dina Boluarte?",
    "¿Qué irregularidades en programas sociales como Qali Warma han implicado al gobierno de Dina Boluarte en casos de corrupción?",
    "¿Qué implicaciones tiene la corrupción detectada en programas sociales como Qali Warma en el gobierno de Dina Boluarte?",
    "¿Qué casos de corrupción relacionados con programas sociales, especialmente Qali Warma, han sido señalados en el contexto del gobierno de Dina Boluarte?"
]


In [ ]:
# ---------------------
# Recolectar resultados
# ---------------------
results_dict = {}

In [ ]:
# -----------------------
# Procesar cada resultado
# -----------------------
for query in variations:
    print(f"Consultando: {query}")
    result_raw = tool.invoke({"query": query})
    if isinstance(result_raw, str):
        try:
            results_list = json.loads(result_raw)
        except Exception as e:
            print(f"Error al parsear JSON para la query '{query}':", e)
            results_list = []
    elif isinstance(result_raw, list):
        results_list = result_raw
    else:
        results_list = []
    for res in results_list:
        url = res.get("url", "").strip()
        if not url:
            continue
        if url in results_dict:
            continue
        snippet = res.get("content", "")
        full_title = obtener_titulo_completo(url)
        full_content_raw = extraer_contenido_newspaper(url)
        full_content = formatear_texto_continuo(full_content_raw) if full_content_raw else ""
        
        record = {
            "query": query,
            "direct_snippet": snippet,
            "url": url,
            "full_title": full_title,
            "full_content": full_content
        }
        results_dict[url] = record
    
    time.sleep(1)

final_results = list(results_dict.values())
print(f"Total de resultados únicos obtenidos: {len(final_results)}")

Consultando: ¿Cómo han respondido los sectores sociales y económicos a las decisiones políticas y económicas de Dina Boluarte?
Consultando: ¿Qué respuestas han mostrado los sectores sociales y económicos frente a las medidas políticas y económicas adoptadas por Dina Boluarte?
Consultando: ¿De qué forma han reaccionado los ámbitos social y económico ante las políticas implementadas por Dina Boluarte?
Consultando: ¿Cuál ha sido la reacción de los sectores social y económico a las decisiones gubernamentales de Dina Boluarte?
Consultando: ¿Cómo se han manifestado las respuestas de la sociedad y del sector económico ante las decisiones políticas y económicas de Dina Boluarte?
Consultando: ¿Qué actitud han tomado los sectores sociales y económicos ante las decisiones en política y economía de Dina Boluarte?
Consultando: ¿Cómo han respondido los diversos sectores sociales y económicos a las medidas adoptadas por el gobierno de Dina Boluarte?
Consultando: ¿Qué reacción se observa en los sector

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Consultando: ¿Cuál es el impacto de la política exterior de Dina Boluarte en la proyección internacional del Perú?
Consultando: ¿Cómo evalúan los analistas internacionales la política exterior de Dina Boluarte y su efecto en la imagen del Perú?
Consultando: ¿Qué repercusiones ha tenido la política exterior de Dina Boluarte en la percepción del Perú a nivel mundial?
Consultando: ¿Cómo ha modificado la política exterior de Dina Boluarte la imagen internacional del Perú?
Consultando: ¿Qué transformaciones en la reputación internacional del Perú se observan tras la política exterior de Dina Boluarte?
Consultando: ¿Qué impacto se ha generado en la imagen global del Perú a partir de la política exterior implementada por Dina Boluarte?
Consultando: ¿De qué forma ha influido la política exterior de Dina Boluarte en la imagen del Perú en el escenario internacional?
Consultando: ¿Qué cambios en la percepción internacional del Perú se relacionan con la política exterior de Dina Boluarte?
Consulta

In [ ]:
# ----------------------------------
# Guardar resultados en JSON y Excel
# ----------------------------------
with open("ResultadosJson.json", "w", encoding="utf-8") as f_json:
    json.dump(final_results, f_json, ensure_ascii=False, indent=2)

df = pd.DataFrame(final_results)
df.to_excel("ResultadosExcel.xlsx", index=False)